### **Parte 2: Tabla Analítica**

**2.1 Creación de la tabla gold_sales_daily_summary**
- Como en el parte 1, se detectaron errores en el calculos del tax y el total_sales_tax. En esta tabla los vamos a recalcular para asegurar que lo que mostremos en los dashboards sea consistente
- Se creó la tabla con los requerimientos solicitados y las metricas calculadas
- Como tenemos transacciones en VES y USD y el comportamiento es similar en ambas,vamos a asumir una tasa de cambio de **1 USD = 1 VES**, de esta manera no se distorcionan los valores totales de la prueba. En un escenario ideal, se tomaria la tasa oficial de cambio.

In [0]:
%sql
-- =========================================
-- Creación de la tabla gold_sales_daily_summary
-- =========================================

CREATE OR REPLACE TABLE workspace.operations.gold_sales_daily_summary AS
SELECT

-- 1. Selección de dimensiones: se incluyen fecha, tienda, producto, proveedor, canal y moneda 
--    para poder segmentar las ventas de forma granular y analizar cualquier combinación de estas dimensiones.

    -- Dimensiones de fecha
    f.BUSINESS_DATE,
    YEAR(f.BUSINESS_DATE) AS YEAR,
    MONTH(f.BUSINESS_DATE) AS MONTH,
    DAY(f.BUSINESS_DATE) AS DAY_OF_MONTH,
    DATE_FORMAT(f.BUSINESS_DATE, 'EEEE') AS DAY_OF_WEEK,

    -- Dimensiones de tienda
    o.STORE_ID AS STORE_ID,
    o.STORE_NAME AS STORE_NAME,
    o.CITY AS STORE_CITY,
    o.REGION_NAME AS STORE_REGION,
    o.FORMAT_NAME AS STORE_FORMAT,

    -- Dimensiones de producto
    p.ITEM AS PRODUCT_CODE,
    p.ITEM_DESC AS PRODUCT_DESC,
    p.DEPT_NAME AS PRODUCT_DEPARTMENT,
    p.CLASS_NAME AS PRODUCT_CLASS,
    p.BRAND_NAME AS PRODUCT_BRAND,

    -- Dimensión proveedor
    s.SUP_NAME AS SUPPLIER_NAME,

    -- Dimensión canal
    c.CHANNEL_NAME AS CHANNEL_NAME,
    c.CHANNEL_TYPE_NAME AS CHANNEL_TYPE,

    -- Dimensión moneda
    m.CURRENCY_ABR AS CURRENCY,

-- 2. Cálculo de métricas de negocio: estas métricas permiten evaluar ventas, rentabilidad, volumen y comportamiento de compra.

    -- Métricas de negocio 
    SUM(f.TOTAL_SALE) AS TOTAL_SALE,
    ROUND(SUM(f.TOTAL_SALE * 0.16), 2) AS TOTAL_TAX,                       -- TAX recalculado
    ROUND(SUM(f.TOTAL_SALE + (f.TOTAL_SALE * 0.16)), 2) AS TOTAL_SALE_TAX,  -- TOTAL_SALE_TAX recalculado
    SUM(f.TOTAL_COST) AS TOTAL_COST,
    SUM(f.TOTAL_SALE - f.TOTAL_COST) AS GROSS_MARGIN,
    CASE WHEN SUM(f.TOTAL_SALE) = 0 THEN 0
         ELSE ROUND(SUM(f.TOTAL_SALE - f.TOTAL_COST)/SUM(f.TOTAL_SALE)*100, 2) END AS PCT_GROSS_MARGIN,
    SUM(f.UNITS) AS TOTAL_UNITS,
    COUNT(f.Q_TRANS) AS TOTAL_TRANSACTIONS,
    CASE WHEN COUNT(f.Q_TRANS) = 0 THEN 0
         ELSE ROUND(SUM(f.TOTAL_SALE + (f.TOTAL_SALE * 0.16))/COUNT(f.Q_TRANS), 2) END AS AVG_TICKET,
    CASE WHEN COUNT(f.Q_TRANS) = 0 THEN 0
         ELSE ROUND(SUM(f.UNITS)/COUNT(f.Q_TRANS), 2) END AS UNITS_PER_TRANSACTION

FROM workspace.operations.gold_fact_sales_product f
-- 3. Joins internos (INNER JOIN) con dimensiones:
--    Se utiliza INNER JOIN para asegurar que solo se incluyan transacciones que tengan correspondencia
--    en todas las dimensiones. Evita registros huérfanos y permite enriquecer la tabla de hechos
INNER JOIN workspace.operations.gold_dim_product p
    ON f.SK_PRODUCT = p.SK_PRODUCT
INNER JOIN workspace.operations.gold_dim_organization o
    ON f.SK_ORGANIZATION = o.SK_ORGANIZATION
INNER JOIN workspace.operations.gold_dim_supplier s
    ON f.SK_SUPPLIER = s.SK_SUPPLIER
INNER JOIN workspace.operations.gold_dim_channel c
    ON f.SK_CHANNEL = c.SK_CHANNEL
INNER JOIN workspace.operations.gold_dim_currency m
    ON f.SK_CURRENCY = m.SK_CURRENCY

-- 4. Agregación (GROUP BY):
--    Se agrupan todas las transacciones que comparten la misma combinación de dimensiones en una sola fila.
--    Evita duplicidad y permite calcular métricas resumidas diarias y por combinación de dimensiones.
GROUP BY
    f.BUSINESS_DATE,
    o.STORE_ID, o.STORE_NAME, o.CITY, o.REGION_NAME, o.FORMAT_NAME,
    p.ITEM, p.ITEM_DESC, p.DEPT_NAME, p.CLASS_NAME, p.BRAND_NAME,
    s.SUP_NAME,
    c.CHANNEL_NAME, c.CHANNEL_TYPE_NAME,
    m.CURRENCY_ABR;


In [0]:
%sql
-- Consultamos la tabla que creamos en el paso anterior
SELECT *
FROM workspace.operations.gold_sales_daily_summary;

**2.2 Validaciones**

**2.2.1 Número total de registros de la tabla creada**
- El número de total de registros de la tabla creada es de 430.910
- 2077 registros fueron agrupados en la creacion de la tabla, por eso da menos que el numero de registros de la tabla original 432.987

In [0]:

%sql
-- Total de registros de la tabla creada
SELECT COUNT(*) AS total_registros
FROM workspace.operations.gold_sales_daily_summary;

In [0]:
%sql
-- =========================================
-- Cantidad de registros que se agruparon al resumir la tabla
-- =========================================
-- Lógica:
-- 1. Se cuenta el total de registros originales en la tabla de hechos.
-- 2. Se cuenta el número de combinaciones únicas de las claves principales utilizadas en el GROUP BY
--    (BUSINESS_DATE, SK_ORGANIZATION, SK_PRODUCT, SK_CHANNEL, SK_CURRENCY, SK_SUPPLIER).
-- 3. Se resta el total de combinaciones únicas del total de registros:
--    - Esto nos da cuántos registros se consolidaron o "agrupados" al hacer la agregación.
--    - Es útil para entender cuántos registros múltiples se resumen en una sola fila en la tabla agregada.
SELECT
    COUNT(*) - COUNT(DISTINCT
        CONCAT(
            BUSINESS_DATE,
            SK_ORGANIZATION,
            SK_PRODUCT,
            SK_CHANNEL,
            SK_SUPPLIER,
            SK_CURRENCY
        )
    ) AS total_rows_grouped
FROM workspace.operations.gold_fact_sales_product;


In [0]:
%sql
-- Cantidad de registros de la tabla original
SELECT COUNT(*) AS total_registros
FROM workspace.operations.gold_fact_sales_product;

**2.2.2 Validar que el Total_Sale siga dando igual que la tabla original**
- Ambos Total_Sale **siguen dando igual**

In [0]:
%sql
-- Validacion de la suma de total_sale en ambas tablas
SELECT 
    (SELECT ROUND(SUM(TOTAL_SALE),2) FROM workspace.operations.gold_fact_sales_product) AS total_sale_original,
    (SELECT ROUND(SUM(TOTAL_SALE),2) FROM workspace.operations.gold_sales_daily_summary) AS total_sale_tabla_analitica;

**2.2.3 Validación de que no hayan valores nulos en campos críticos**
- No se encontraron valores nulos en campos críticos

In [0]:
%sql
-- Se validan campos críticos (fecha, tienda, producto, proveedor, moneda, total_sale, units, transactions) para asegurar integridad y segmentación correcta de los registros.

SELECT
    SUM(CASE WHEN BUSINESS_DATE IS NULL THEN 1 ELSE 0 END) AS null_business_date,
    SUM(CASE WHEN STORE_NAME IS NULL THEN 1 ELSE 0 END) AS null_store_name,
    SUM(CASE WHEN PRODUCT_CODE IS NULL THEN 1 ELSE 0 END) AS null_product_code,
    SUM(CASE WHEN PRODUCT_DEPARTMENT IS NULL THEN 1 ELSE 0 END) AS null_product_department,
    SUM(CASE WHEN PRODUCT_BRAND IS NULL THEN 1 ELSE 0 END) AS null_product_brand,
    SUM(CASE WHEN SUPPLIER_NAME IS NULL THEN 1 ELSE 0 END) AS null_supplier_name,
    SUM(CASE WHEN CURRENCY IS NULL THEN 1 ELSE 0 END) AS null_currency,
    SUM(CASE WHEN TOTAL_SALE IS NULL THEN 1 ELSE 0 END) AS null_total_sale,
    SUM(CASE WHEN TOTAL_UNITS IS NULL THEN 1 ELSE 0 END) AS null_total_units,
    SUM(CASE WHEN TOTAL_TRANSACTIONS IS NULL THEN 1 ELSE 0 END) AS null_total_transactions
FROM workspace.operations.gold_sales_daily_summary;